In [1]:
import numpy as np
import pandas as pd
import os
import re
import csv

if not os.path.exists("data/moldes"):
    os.makedirs("data/moldes")

In [2]:
def create_molde_vagas():
    print("Creating molde_vagas.csv ...")
    
    # Readind vagas paths and storing
    b_path = "raw_data/arquivos_vagas/unziped/"
    paths = os.listdir(b_path)
    consulta_vagas = [k for k in paths if "csv" in k or 'txt' in k]
    
    print("Tamanho da lista detalhe_votacao: \n{}".format(len(consulta_vagas)))
    
    # Defining Headers
    headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO","DESCRICAO_ELEICAO", "SIGLA_UF",
           "SIGLA_UE", "NOME_UE", "CODIGO_CARGO", "DESCRICAO_CARGO", "QTDE_VAGAS"]
    
    # Reading data
    data = []
    for a in consulta_vagas:
        # 2014 special case
        if '2014' in a:
            skiprows_use=1
        else:
            skiprows_use=None

        data.append(pd.read_csv(b_path + a, sep = ';', encoding='latin_1', header=None, skiprows=skiprows_use))
    
    # Fixing columns lenght and defining columns names
    for i in range(len(data)):
        if len(data[i].columns) == 10:
            data[i].columns = headers
        else:
            data[i] = data[i].iloc[:,[0,1,2,6,9,10,11,12,13,14]]
            data[i].columns = headers
    
    # Bindng all data
    data = pd.concat(data)
    
    # Droping obs
    data = data[np.in1d(data.DESCRICAO_ELEICAO, ['Eleição Suplementar Governador AM', 'Eleição Suplementar Governador - TO'], invert=True)]
    
    # Droping coluns which will not be used
    data = data.iloc[:,2:len(data.columns)].drop(["SIGLA_UF", 'DESCRICAO_ELEICAO'], axis = 1)
    
    # Creating by hand president DataFrame
    vagas_presidente = {'ANO_ELEICAO': np.arange(1998,2018,4),
                    'SIGLA_UE': "BR",
                    'NOME_UE': "Brasil",
                    'CODIGO_CARGO': 1,
                    'DESCRICAO_CARGO': 'Presidente',
                    'QTDE_VAGAS': 1}
    
    presidente_df = pd.DataFrame(vagas_presidente)
    
    # Binding president and data together

    data = pd.concat([data, presidente_df])
    
    print("Writing data into molde_vagas.csv")
    
    data.to_csv("data/moldes/molde_vagas.csv")
    
    return data

In [3]:
create_molde_vagas().head()

Creating molde_vagas.csv ...
Tamanho da lista detalhe_votacao: 
166
Writing data into molde_vagas.csv


,ANO_ELEICAO,SIGLA_UE,NOME_UE,CODIGO_CARGO,DESCRICAO_CARGO,QTDE_VAGAS
0,2014,MG,MINAS GERAIS,3,Governador,1
1,2014,MG,MINAS GERAIS,4,Vice-governador,1
2,2014,MG,MINAS GERAIS,5,Senador,1
3,2014,MG,MINAS GERAIS,6,Deputado Federal,53
4,2014,MG,MINAS GERAIS,7,Deputado Estadual,77


In [10]:
def create_molde_secao():
    print("Creating molde_vagas.csv:")

    # Readind vagas paths and storing
    b_path = "raw_data/arquivos_secao/unziped/"
    paths = os.listdir(b_path)
    arquivos_secao = [k for k in paths if "csv" in k or 'txt' in k]

    # Defining Headers
    headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO", "NUM_TURNO", "DESCRICAO_ELEICAO",
               "SIGLA_UF", "SIGLA_UE", "CODIGO_MUNICIPIO", "NOME_MUNICIPIO","NUM_ZONA","NUM_SECAO",
               "CODIGO_CARGO", "DESCRICAO_CARGO", "QTD_APTOS", "QTD_COMPARECIMENTO", "QTD_ABSTENCOES", 
               "QT_VOTOS_NOMINAIS", "QT_VOTOS_BRANCOS", "QT_VOTOS_NULOS", "QT_VOTOS_LEGENDA",
               "QT_VOTOS_ANULADOS_APU_SEP"]
    
    data = []
    for ano in np.arange(1998,2018,4):
        print("Parsing data from " + str(ano) + "...")
        data_spec = []
        for file_path in arquivos_secao:
            if str(ano) in file_path:
                data_spec.append(pd.read_csv(b_path + file_path, sep = ';', header=None, encoding="latin-1"))

        data_spec = pd.concat(data_spec)
        data_spec.columns = headers

        data_spec = data_spec.loc[:,['ANO_ELEICAO',
                                     'NUM_TURNO',
                                     'DESCRICAO_ELEICAO',
                                     'SIGLA_UE',
                                     'CODIGO_MUNICIPIO',
                                     'NOME_MUNICIPIO',
                                     'CODIGO_CARGO',
                                     'DESCRICAO_CARGO',
                                     'QTD_APTOS',
                                     'QTD_COMPARECIMENTO']]

        data_spec = data_spec.groupby(['ANO_ELEICAO',
                                       'NUM_TURNO',
                                       'DESCRICAO_ELEICAO',
                                       'SIGLA_UE',
                                       'CODIGO_MUNICIPIO',
                                       'NOME_MUNICIPIO',
                                       'CODIGO_CARGO',
                                       'DESCRICAO_CARGO',])

        data_spec = data_spec.sum().reset_index()

        data.append(data_spec)
        
    data = pd.concat(data)
    
    for ano in np.arange(2010,1998,-8):
        temp = data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]]
        temp = temp * 2
        data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp
    
    temp = data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]]
    
    temp = temp / 2
    
    data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp
    
    data.to_csv("data/moldes/molde_secao.csv")
    return data
    

In [11]:
data = create_molde_secao()

Creating molde_vagas.csv:
Parsing data from 1998...


/home/rafael/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (2,3,7,9,10,11,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Parsing data from 2002...
Parsing data from 2006...
Parsing data from 2010...
Parsing data from 2014...


In [12]:
data[data.ANO_ELEICAO == 2002].head()

,ANO_ELEICAO,NUM_TURNO,DESCRICAO_ELEICAO,SIGLA_UE,CODIGO_MUNICIPIO,NOME_MUNICIPIO,CODIGO_CARGO,DESCRICAO_CARGO,QTD_APTOS,QTD_COMPARECIMENTO
0,2002,1,ELEICOES 2002,19,19,GUAJARA MIRIM,1,PRESIDENTE,22458,17436
1,2002,1,ELEICOES 2002,19,19,GUAJARA MIRIM,3,GOVERNADOR,22458,17436
2,2002,1,ELEICOES 2002,19,19,GUAJARA MIRIM,5,SENADOR,44916,34872
3,2002,1,ELEICOES 2002,19,19,GUAJARA MIRIM,6,DEPUTADO FEDERAL,22458,17436
4,2002,1,ELEICOES 2002,19,19,GUAJARA MIRIM,7,DEPUTADO ESTADUAL,22458,17436
